In [3]:
import tensorflow as tf

### tf.Variableとtf.constant

In [3]:
# constantは定数
a = tf.constant(3, name="const1")
# Variableは変数(objectも可)
b = tf.Variable(0, name="val1")
c = tf.add(a, b)

### tf.placeholder()
- 宣言時には値が分からないもの

```
assign = tf.assign(b, add)
c = tf.placeholder(tf.int32, name="input")
mul = tf.mul(d, e)
```

- 初期化時に型だけ決めておき実行時にdictionary型で実際の値を与える

### 変数の初期化
- tf.global_variables_initializerで初期化する

### tf.Session()で実行する


### tf.reduce_mean()
- テンソルの平均値を計算する
- numpyのnp.mean()と同じ
- 第２引数にaxisをとる

In [24]:
import tensorflow as tf

a = [[1., 2., 3.], [4., 5., 6.]]

x = tf.placeholder(tf.float32, [2, 3])
y_1 = tf.reduce_mean(x, axis=0)
y_2 = tf.reduce_mean(x, axis=1)
y_3 = tf.reduce_mean(x)

with tf.Session() as sess:
    print(a)
    print(sess.run(y_1, feed_dict={x: a}))
    print(sess.run(y_2, feed_dict={x: a}))
    print(sess.run(y_3, feed_dict={x: a}))

[[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]]
[2.5 3.5 4.5]
[2. 5.]
3.5


- axis(軸)が0だと列に対して平均値を出す
- axis(軸)が1だと行に対して平均値を出す
- axis(軸)が指定なしだと全ての値の平均値をだす

MNISTを使った入門
----------------------

In [10]:
import pandas as pd
# mnistはtensorflowにあるexampleを使う
# (バイナリからベクトルに変換するのが大変なので)
from tensorflow.examples.tutorials.mnist import input_data
import tensorflow as tf

In [15]:
# mnistデータを格納したオブジェクトを呼び出す
mnist = input_data.read_data_sets("data/", one_hot=True)

# 全訓練データの取得

# 訓練用の入力データ、正解データをミニバッチ数を指定して取得
train_images, train_labels = mnist.train.next_batch(50)

# テスト用の全画像データを取得
test_images = mnist.test.images
# テスト用の全正解データを取得
test_labels = mnist.test.labels

Extracting data/train-images-idx3-ubyte.gz
Extracting data/train-labels-idx1-ubyte.gz
Extracting data/t10k-images-idx3-ubyte.gz
Extracting data/t10k-labels-idx1-ubyte.gz
<class 'numpy.ndarray'>


### 順伝播

In [30]:
# 入力データを定義

# モデル構築時にどの画像を渡すか決められないので、tf.placeholder()を使って実行時に引数で渡す
# 画素値は正規化してあるため浮動小数点=>float32
# 入力は[ミニバッチサイズ、１枚の画像の画素数]
## Noneに指定するのは、入力時に自動で計算するため値はなんでもいいという意味
x = tf.placeholder(tf.float32, [None, 784])

In [31]:
# 入力層から中間層

# 中間層のユニット数は64にする
# 学習パラメタ、バイアスともに確率勾配法を用いてアップデートされる変数なのでtf.Variableで宣言
w_1 = tf.Variable(tf.truncated_normal([784, 64], stddev=0.1), name="w1")
b_1 = tf.Variable(tf.zeros([64]), name="b1")
h_1 = tf.nn.relu(tf.matmul(x, w_1) + b_1)

- truncated_normalは正規分布をもとに指定したshapeの値を自動生成するオペレーション
    - ニューラルネットワークの重みを初期化するときには基本的にこれを使用する
- tf.Variable(tf.zeros([64]), name="b1")
    - バイアスは一般的に0で初期化する
    - shapeが合わないがブロードキャスティング機能で演算されるため問題ない
        - ※ブロードキャスティング機能: shapeが合わないテンソル同士の演算で階層が多い方のshapeに合わせて上手く演算してくれる
- tf.matmul(x, w_1)で行列の積を出しReLUにかける

In [32]:
# 中間層から出力層

w_2 = tf.Variable(tf.truncated_normal([64, 10], stddev=0.1), name="w2")
b_2 = tf.Variable(tf.zeros([10]), name="b2")
out = tf.nn.softmax(tf.matmul(h_1, w_2) + b_2)

### 誤差関数、訓練

In [33]:
# 正解ラベルも実行時に渡せるようにplaceholderにする
y = tf.placeholder(tf.float32, [None, 10])
# 二乗誤差関数を採用
loss = tf.reduce_mean(tf.square(y - out))

In [34]:
# 訓練
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

- GradientDescentOptimizerは確率的勾配降下法の実装オペレーション
    - 引数の0.5は学習率を表す

In [35]:
# 評価
correct = tf.equal(tf.argmax(out, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

- tf.argmaxで正解データと出力データで一番大きな値を求める
    - 引数1はaxisのため行に対して最大値を求めている
- tf.equalで同じラベルを指しているかを判定する
- Trueの割合を求めることで正解率を出している

In [36]:
# 変数の初期化
init = tf.global_variables_initializer()

### 実行

In [40]:
with tf.Session() as sess:
    sess.run(init)
    
    #　テストデータのロード
    test_images = mnist.test.images
    test_labels = mnist.test.labels
    
    for i in range(1000):
        step = i + 1
        train_images, train_labels = mnist.train.next_batch(50)
        sess.run(train_step, feed_dict={x: train_images, y: train_labels})
        
        if step % 10 == 0:
            acc_val = sess.run(accuracy, feed_dict={x: test_images, y: test_labels})
            print("Step {:d}: accuracy = {:.2}".format(step, acc_val))

Step 10: accuracy = 0.14
Step 20: accuracy = 0.17
Step 30: accuracy = 0.2
Step 40: accuracy = 0.23
Step 50: accuracy = 0.27
Step 60: accuracy = 0.32
Step 70: accuracy = 0.36
Step 80: accuracy = 0.4
Step 90: accuracy = 0.41
Step 100: accuracy = 0.45
Step 110: accuracy = 0.45
Step 120: accuracy = 0.47
Step 130: accuracy = 0.51
Step 140: accuracy = 0.53
Step 150: accuracy = 0.55
Step 160: accuracy = 0.59
Step 170: accuracy = 0.6
Step 180: accuracy = 0.61
Step 190: accuracy = 0.64
Step 200: accuracy = 0.65
Step 210: accuracy = 0.69
Step 220: accuracy = 0.72
Step 230: accuracy = 0.73
Step 240: accuracy = 0.74
Step 250: accuracy = 0.75
Step 260: accuracy = 0.77
Step 270: accuracy = 0.77
Step 280: accuracy = 0.77
Step 290: accuracy = 0.79
Step 300: accuracy = 0.8
Step 310: accuracy = 0.81
Step 320: accuracy = 0.81
Step 330: accuracy = 0.81
Step 340: accuracy = 0.82
Step 350: accuracy = 0.82
Step 360: accuracy = 0.82
Step 370: accuracy = 0.83
Step 380: accuracy = 0.83
Step 390: accuracy = 0.83